In [ ]:
from __future__ import print_function
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import time

In [ ]:
# with open("data/shakespeare.txt", "r") as data_file:
with open("data/tinyshakespeare.txt", "r") as data_file:
    data = data_file.read()

In [ ]:
data_size = len(data)
print(data_size)

In [ ]:
alphabet = set(data)
alphabet_size = len(alphabet)
print(alphabet_size)

In [ ]:
symbol_to_id = {}
id_to_symbol = {}
for symbol_id, symbol in enumerate(sorted(alphabet)):
    symbol_to_id[symbol] = symbol_id
    id_to_symbol[symbol_id] = symbol

In [ ]:
def one_hot_encoding(symbol):
    one_hot_encoded = torch.zeros(alphabet_size)
    symbol_id = symbol_to_id[symbol]
    one_hot_encoded[symbol_id] = 1
    return one_hot_encoded

In [ ]:
def labels_tensor(symbols):
    return torch.Tensor([symbol_to_id[symbol] for symbol in symbols]).long()

In [ ]:
hidden_size = 100

class MinCharRNN(nn.Module):
    
    def __init__(self):
        super(MinCharRNN, self).__init__()
        
        self.input_to_hidden = nn.Linear(alphabet_size, hidden_size)
        self.hidden_to_hidden = nn.Linear(hidden_size, hidden_size)
        self.hidden_to_output = nn.Linear(hidden_size, alphabet_size)

    def forward(self, input_symbol, hidden_state):
        hidden_state = F.tanh(self.input_to_hidden(input_symbol) + self.hidden_to_hidden(hidden_state))
        output = self.hidden_to_output(hidden_state)
        return output, hidden_state

In [ ]:
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.uniform(m.weight, -0.01, 0.01)

In [ ]:
learning_rate = 1e-1

model = MinCharRNN()    
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(model.parameters(), lr=learning_rate)

model.apply(initialize_weights)

In [ ]:
# model.load_state_dict(torch.load("data/min-char-rnn.torch"))

In [ ]:
sample_size = 200
first_symbol = "\n"

def print_sample():
    v_input_symbol = Variable(one_hot_encoding(first_symbol), volatile=True)
    v_hidden_state = Variable(torch.zeros((1, hidden_size)), volatile=True)

    sample = ""
    
    for sample_id in range(sample_size):
        v_logits, v_hidden_state = model(v_input_symbol, v_hidden_state)
        probabilities = F.softmax(v_logits).data.squeeze(0).numpy()
        symbol_id = np.random.choice(range(alphabet_size), p=probabilities)
        symbol = id_to_symbol[symbol_id]
        sample += symbol
        v_input_symbol = Variable(one_hot_encoding(symbol), volatile=True)
    
    print("-" * 80)
    print(sample)
    print("-" * 80)

In [ ]:
print_sample()

In [ ]:
epochs = 10
sequence_size = 25
batches = data_size // (sequence_size + 1)
log_every = 10000
gradient_clipping = 5

initial_state = torch.zeros((1, hidden_size))

for epoch_id in range(epochs):
    # reset the state before every epoch
    last_hidden_state = initial_state
    
    accumulated_loss = 0.0
    start_time = time.time()
    
    epoch_accumulated_loss = 0.0
    epoch_start_time = time.time()
    
    for batch_id in range(batches):
        batch_start = batch_id * sequence_size
        
        # reuse the hidden state from last batch
        hidden_state = Variable(last_hidden_state)
        
        # clear the gradient information from the past batch
        optimizer.zero_grad()
        
        # for every symbol in the batch
        # try predict the next symbol
        # and meassure the loss
        predictions = []
        for sequence_id in range(sequence_size):
            v_input_symbol = Variable(one_hot_encoding(data[batch_start + sequence_id]))
            
            v_prediction, hidden_state = model(v_input_symbol, hidden_state)
            
            predictions.append(v_prediction)
            
        # create all labels
        v_labels = Variable(labels_tensor(data[batch_start + 1:batch_start + sequence_size + 1]))
        
        # create all predictions
        v_predictions = torch.cat(predictions)
            
        # backpropagate through time
        v_loss = loss_function(v_predictions, v_labels)
        v_loss.backward()
        
        # gradient clipping to avoid exploding gradients
        for parameter in model.parameters():
            parameter.grad.data.clamp_(-gradient_clipping, gradient_clipping)
        
        # update parameters
        optimizer.step()
        
        # some logging
        accumulated_loss += v_loss.data[0]
        epoch_accumulated_loss += v_loss.data[0]
        
        if batch_id % log_every == log_every - 1:
            mean_loss = accumulated_loss / float(log_every)
            end_time = time.time()
            elapsed_time = end_time - start_time
            accumulated_loss = 0.0
            start_time = end_time
            print("Batch: {:6d} Loss: {:.4f} Time: {:.2f} seconds".format(batch_id + 1, mean_loss, elapsed_time))
        
        # take the hidden state out of the variable
        # to avoid backpropagating the next batch to this one
        last_hidden_state = hidden_state.data
        
    mean_loss = epoch_accumulated_loss / float(batches)
    elapsed_time = time.time() - epoch_start_time

    print("Epoch: {:6d} Loss: {:.4f} Time: {:.2f} seconds".format(epoch_id + 1, mean_loss, elapsed_time))

    print_sample()

In [ ]:
# torch.save(model.state_dict(), "data/min-char-rnn.torch")